<center><h1 style="color:red;" >Analysis of AirBnb Data</h1></center>

# Introduction

<p>AirBnb is one of the fastest growing companies. It provides a service where hosts with spare rooms can rent them out to guests and collects a comission. More details about the company can be found at <a href="https://www.airbnb.com/">https://www.airbnb.com/</a> </p>.
<p>Our interest is finding out what their data can tell us. What insights we can get and if this insights can help other businesses in the same industry s</p>